# Evaluating your Amazon Bedrock Agent with Ragas

## Introduction

This notebook demonstrates a Restaurant Booking Agent built with Amazon Bedrock Agents and RAGAS evaluation framework. 
We build a restaurant Booking agent, with two primary functions: 
1. Retrieving menu information from an Amazon Bedrock Knowledge Base
2. Processing table reservations through an Amazon DynamoDB database. 

When customers make requests, the agent automatically determines which tools to use and executes the appropriate actions through AWS Lambda functions via action groups. 

The evaluation provides quantitative measures of how well our agent performs its tasks, helping identify strengths and areas for improvement.

#### Agent Architecture 
This is how our Agent architecture looks like:

<div style="text-align:left">
    <img src="images/architecture.png" width="65%" />
</div>



## Setup

Let's install the required libraries, including latest boto3 and ragas python package.

In [ ]:
!pip install --upgrade -q -r requirements.txt

## Creating Agent

First, we declare global variables that will be act as helpers for the entire notebook. After that, we will start to create our Agent.

In [ ]:
import boto3
import os
import uuid
import time

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"
s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'us.anthropic.claude-3-5-sonnet-20240620-v1:0',
    'us.anthropic.claude-3-5-haiku-20241022-v1:0'
]

In [ ]:
# Define variables for the agent and its resources
agent_name = f"rest-{agent_suffix}"
lambda_name = f"rest-{agent_suffix}"

agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

dynamodb_table = f"{agent_name}-table"
dynamodb_pk = "booking_id"
dynamodb_sk = "restaurant_name"

dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]

knowledge_base_name = f'{agent_name}-kb'
suffix = f"{region}-{account_id}"

knowledge_base_description = "KB to reply to questions about the restaurants and their menus"
bucket_name = f'restaurant-kb-{suffix}'

### Importing helper functions

In the following section, we're adding `bedrock_agent_helper.py` and `knowledge_base_helper.py` on Python path, so that these two files can be recognized, and their functionalities can be used by importing the helper classes.

Those two helper classes handle the interactions with Amazon Bedrock. Using the helper classes, you'd import the following methods in this lab:

In `bedrock_agent_helper`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

In `knowledge_base_helper` :
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [ ]:
import sys
import sys
import os

# Get the current working directory and navigate to the project root
current_dir = os.path.dirname(os.path.abspath(
    __file__ if '__file__' in globals() else os.getcwd()
))

# Navigate up to the project root 
project_root = os.path.join(current_dir, '..', '..')
project_root = os.path.abspath(project_root)

# Add the src directory to Python path
src_path = os.path.join(project_root, 'src')
sys.path.insert(0, src_path)

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Create and Load Knowledge Base

In this section, let's create a Amazon Bedrock Knowledge Base and ingest data in it. 

In the next steps we will generate the data used to populate the knowledge base. It will be composed of instructions on how to handle a solar panel

**Note: This creation process takes several minutes.**

In [ ]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

### Create Synthetic Data to Load on S3

Instead of get data elsewhere, Let's generate synthetic data, using a LLM on Amazon Bedrock.
This synthetic data that will be generated, will be uploaded into a S3 bucket and then added into a **Amazon Bedrock Knowledge Base**.

In [ ]:
path = "kb_documents"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
   # Create a n ew directory if it does not exist
   os.makedirs(path)
   print("The {} directory was created!".format(path))
else:
   print("The {} directory already exists!".format(path))

### Upload data to S3
Uploading generated files into S3 Bucket.

In [ ]:
def upload_directory(path, bucket_name):
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,file)

### Synchronize with the Knowledge Base
Now that the data is available in the s3 bucket, let's synchronize it to our knowledge base

In [ ]:
upload_directory("kb_documents", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

## Create Agent

Now that we have the data setup, lets start creating the Restaurant Agent that will have a **Knowledge Base** and a **Lambda Action Group** to handle exception workflow (when information is not found on KB).

For this agent we will use the following instructions:
```
  You are a "Restaurant Helper", a restaurant assistant helping customers reserving tables in 
  different restaurants. You can talk about the menus, create new bookings, get the details of an existing booking 
  or delete an existing reservation. You reply always politely and mention your name in the reply (Restaurant Helper). 
  NEVER skip your name in the start of a new conversation. If customers ask about anything that you cannot reply, 
  please provide the following phone number for a more personalized experience: +1 999-999-9999.
  
  Some information that will be useful to answer your customer's questions:
  Restaurant Helper Address: 101W 87th Street, 100024, New York, New York
  You should only contact restaurant helper for technical support.
  Before making a reservation, make sure that the restaurant exists in our restaurant directory.
  
  ALWAYS use the greeting agent to say hi in the first conversation.
```


We will also connect a knowledge base with the information about Restaurant menus with the following instructuctions:

```
Use the knowledge base retrieval to reply to questions about the restaurants and their menus.
```


In [ ]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [ ]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': 'Use the knowledge base retrieval to reply to questions about the restaurants and their menus.'
}

In [ ]:
agent_instruction = """You are \"Restaurant Helper\", a restaurant assistant helping customers reserving tables in 
  different restaurants. You can talk about the menus, create new bookings, get the details of an existing booking 
  or delete an existing reservation. You reply always politely and mention your name in the reply (Restaurant Helper). 
  NEVER skip your name in the start of a new conversation. If customers ask about anything that you cannot reply, 
  please provide the following phone number for a more personalized experience: +1 999 999 99 9999.

  Some information that will be useful to answer your customer's questions:
  Restaurant Helper Address: 101W 87th Street, 100024, New York, New York
  You should only contact restaurant helper for technical support.
  Before making a reservation, make sure that the restaurant exists in our restaurant directory.

  ALWAYS use the greeting agent to say hi in the first conversation."""

agent_description = """Restaurant Assistant agent"""

restaurant_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn]
)
restaurant_agent

In [ ]:
time.sleep(60)
agents.associate_kb_with_agent(
    restaurant_agent[0], kb_config['kb_instruction'], kb_config['kb_id']
)

### Create Lambda Action Group for the Agent

In this section, we're going create an action group to handle restaurant booking and associate it with our agent. To do so:

1. We create a Lambda function code to fulfill the execution of the agent's actions. 
2. We define the actions available actions that an agent can take using function details. You can also define the actions available using OpenAPI Schema.
3. Associate the lambda action group to the agent

#### Step 1: Create Lambda function
The agent is equipped with three key actions defined in the Action Group:
- `get_booking_details`: to get the relevant details for booking_id in restaurant_name
- `create_booking`: to create a new booking at restaurant_name
- `delete_booking`: to delete an existing booking_id at restaurant_name

These actions are implemented through a **Lambda function** that interacts with a DynamoDB table to store and retrieve booking information.

In [ ]:
%%writefile lambda_function.py
import json
import uuid
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('restaurant_bookings')

def get_named_parameter(event, name):
    """
    Get a parameter from the lambda event
    """
    return next(item for item in event['parameters'] if item['name'] == name)['value']


def get_booking_details(booking_id):
    """
    Retrieve details of a restaurant booking
    
    Args:
        booking_id (string): The ID of the booking to retrieve
    """
    try:
        response = table.get_item(Key={'booking_id': booking_id})
        if 'Item' in response:
            return response['Item']
        else:
            return {'message': f'No booking found with ID {booking_id}'}
    except Exception as e:
        return {'error': str(e)}


def create_booking(date, name, hour, num_guests):
    """
    Create a new restaurant booking
    
    Args:
        date (string): The date of the booking
        name (string): Name to idenfity your reservation
        hour (string): The hour of the booking
        num_guests (integer): The number of guests for the booking
    """
    try:
        booking_id = str(uuid.uuid4())[:8]
        table.put_item(
            Item={
                'booking_id': booking_id,
                'date': date,
                'name': name,
                'hour': hour,
                'num_guests': num_guests
            }
        )
        return {'booking_id': booking_id}
    except Exception as e:
        return {'error': str(e)}


def delete_booking(booking_id):
    """
    Delete an existing restaurant booking
    
    Args:
        booking_id (str): The ID of the booking to delete
    """
    try:
        response = table.delete_item(Key={'booking_id': booking_id})
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            return {'message': f'Booking with ID {booking_id} deleted successfully'}
        else:
            return {'message': f'Failed to delete booking with ID {booking_id}'}
    except Exception as e:
        return {'error': str(e)}
    

def lambda_handler(event, context):
    # get the action group used during the invocation of the lambda function
    actionGroup = event.get('actionGroup', '')
    
    # name of the function that should be invoked
    function = event.get('function', '')
    
    # parameters to invoke function with
    parameters = event.get('parameters', [])

    if function == 'get_booking_details':
        booking_id = get_named_parameter(event, "booking_id")
        if booking_id:
            response = str(get_booking_details(booking_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing booking_id parameter'}}

    elif function == 'create_booking':
        date = get_named_parameter(event, "date")
        name = get_named_parameter(event, "name")
        hour = get_named_parameter(event, "hour")
        num_guests = get_named_parameter(event, "num_guests")

        if date and hour and num_guests:
            response = str(create_booking(date, name, hour, num_guests))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing required parameters'}}

    elif function == 'delete_booking':
        booking_id = get_named_parameter(event, "booking_id")
        if booking_id:
            response = str(delete_booking(booking_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing booking_id parameter'}}

    else:
        responseBody = {'TEXT': {'body': 'Invalid function'}}

    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response

#### Step 2 : Define available actions
Now it's time to define the actions that can be taken by the agent

In [ ]:
agent_functions = [
    {
        'name': 'get_booking_details',
        'description': 'Retrieve details of a restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to retrieve",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'create_booking',
        'description': 'Create a new restaurant booking',
        'parameters': {
            "date": {
                "description": "The date of the booking in the format YYYY-MM-DD",
                "required": True,
                "type": "string"
            },
            "name": {
                "description": "Name to idenfity your reservation",
                "required": True,
                "type": "string"
            },
            "hour": {
                "description": "The hour of the booking in the format HH:MM",
                "required": True,
                "type": "string"
            },
            "num_guests": {
                "description": "The number of guests for the booking",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'delete_booking',
        'description': 'Delete an existing restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to delete",
                "required": True,
                "type": "string"
            }
        }
    },
]

#### Step 3: Associating action group to agent
Finally, we can associate a new action group with our previously created agent

In [ ]:
resp = agents.add_action_group_with_lambda(
    agent_name=agent_name,
    lambda_function_name=lambda_name,
    source_code_file="lambda_function.py",
    agent_functions=agent_functions,
    agent_action_group_name="restaurant_helper_actions",
    agent_action_group_description="Actions for getting table booking information, create a new booking or delete an existing booking",
    dynamo_args=dynamoDB_args
)

In [ ]:
restaurant_agent

#### Congratulations, you have created the agent !!!

## Test the Agent

Now, let's run some tests on the agent to make sure it's working. To do so we will use our test alias: `TSTALIASID` which allows you to invoke a draft version of your agent.

### Testing with a simple question
First let's ask a question related to food options in Seattle 

In [ ]:
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
def invokeAgent(query, session_id, session_state=dict()):
    # invoke the agent API
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=restaurant_agent[0],
        agentAliasId=restaurant_agent[1],
        sessionId=session_id,
        enableTrace=True,
        sessionState=session_state,
    )

    event_stream = agentResponse["completion"]
    try:
        traces = []
        for event in event_stream:
            if "chunk" in event:
                data = event["chunk"]["bytes"]
                agent_answer = data.decode("utf8")
                return agent_answer, traces
                # the chunk object indicates that the request finished successfully
            elif "trace" in event:
                traces.append(event["trace"])
                # captures the trace information
            else:
                raise Exception("unexpected event.", event)
        return agent_answer, traces
    except Exception as e:
        raise Exception("unexpected event.", e)

In [ ]:
session_id = str(uuid.uuid1())
query = "Where can I eat in Seattle?"
agent_answer, traces = invokeAgent(query, session_id)
agent_answer

### Evaluate Performance with Ragas

Now we know that the agent works, how do we know that it works well ? 

In this section, lets assess the performance using [Ragas Agent Evaluation Metrics](https://docs.ragas.io/en/stable/howtos/integrations/amazon_bedrock/?h=agentgoal#evaluating-agent-with-ragas).

#### Key Steps
1. Set up Ragas evaluation metrics
2. Convert agent interaction traces to Ragas-compatible format
3. Evaluate the agent using various metrics
4. Analyze the results to understand agent performance

#### 1. Ragas Setup
We configure Ragas with:
- A judge model *(we chose Amazon Bedrock's Nova Pro model)*
- Evaluation metrics for different aspects of agent performance
- Custom evaluation criteria for specific use cases


In [ ]:
from langchain_aws import ChatBedrock
from ragas.llms import LangchainLLMWrapper

# Configure the judge model
model_id = "us.amazon.nova-pro-v1:0"   # Choose your desired model
region_name = "us-east-1"              # Choose your desired AWS region

# Initialize the Bedrock LLM and wrap it for Ragas
bedrock_llm = ChatBedrock(model_id=model_id, region_name=region_name)
evaluator_llm = LangchainLLMWrapper(bedrock_llm)

We start with asking ourselves a major question: 

<div style="text-align:left">
    <img src="images/metrics_1.png" width="65%" />
</div>


For the first question, we choose two metrics:
1. **AgentGoalAccuracyWithoutReference**: LLM as a judge makes the decision
2. **AgentGoalAccuracyWithReference**: Needs a baseline reference / Ground truth.

In [ ]:
from ragas.metrics import (
    AgentGoalAccuracyWithoutReference,
    AgentGoalAccuracyWithReference,
)

# Initialize the goal accuracy metrics
goal_accuracy_with_reference = AgentGoalAccuracyWithReference(llm=evaluator_llm)
goal_accuracy_without_reference = AgentGoalAccuracyWithoutReference(llm=evaluator_llm)

In [ ]:
session_id = str(uuid.uuid1())
query = "Make a reservation for Anna, 2 at the The Coastal Bloom. For August 12th 2025 at 9pm"
agent_answer, traces = invokeAgent(query, session_id)
agent_answer

In [ ]:
from ragas.integrations.amazon_bedrock import convert_to_ragas_messages
from ragas.dataset_schema import SingleTurnSample, MultiTurnSample, EvaluationDataset
from ragas import evaluate

# Convert traces to Ragas format
ragas_messages = convert_to_ragas_messages(traces)

# Test with reference comparison
sample_with_ref = MultiTurnSample(
    user_input=ragas_messages,
    reference="Reservation is done and the booking id is shared",
)

result_with_ref = evaluate(
    dataset=EvaluationDataset(samples=[sample_with_ref]),
    metrics=[goal_accuracy_with_reference],
)

# Test without reference
sample_without_ref = MultiTurnSample(user_input=ragas_messages)

result_without_ref = evaluate(
    dataset=EvaluationDataset(samples=[sample_without_ref]),
    metrics=[goal_accuracy_without_reference],
)

In [ ]:
result_with_ref

In [ ]:
result_without_ref

Next, we want to evaluate the performance of the **Knowledge Retrieval**. We use the following RAG-specific metrics:


<div style="text-align:left">
    <img src="images/metrics_2.png" width="65%" />
</div>


1. **ContextRelevance**: Measures how well the retrieved contexts address the user’s query by evaluating their pertinence through dual LLM judgments.
2. **Faithfulness**: Assesses the factual consistency of the response by determining whether all its claims can be supported by the provided retrieved contexts.
3. **ResponseGroundedness**: Determines the extent to which each claim in the response is directly supported or “grounded” in the provided contexts.

In [ ]:
from ragas.metrics import ContextRelevance, Faithfulness, ResponseGroundedness

# Initialize RAG evaluation metrics
metrics = [
    ContextRelevance(llm=evaluator_llm),
    Faithfulness(llm=evaluator_llm),
    ResponseGroundedness(llm=evaluator_llm),
]

In [ ]:
# first query
query = "Does Rice & Spice in San Francisco serve Chicken Wings?"
agent_answer_wings, traces_wings = invokeAgent(query, str(uuid.uuid1()))

# second query
query = "Does Rice & Spice in San Francisco have Mango Sticky Rice in the menu?"
agent_answer_dessert, traces_dessert = invokeAgent(query, str(uuid.uuid1()))

In [ ]:
print(agent_answer_wings)
print("----")
print(agent_answer_dessert)

In [ ]:
from ragas.integrations.amazon_bedrock import extract_kb_trace

# Extract knowledge base traces from previous agent interactions
kb_trace_wings = extract_kb_trace(traces_wings)
kb_trace_dessert = extract_kb_trace(traces_dessert)

# Create evaluation samples
wings_sample = SingleTurnSample(
    user_input=kb_trace_wings[0].get("user_input"),
    retrieved_contexts=kb_trace_wings[0].get("retrieved_contexts"),
    response=kb_trace_wings[0].get("response"),
    reference="""No, Rice & Spice in San Francisco does not serve Chicken Wings.
    Their menu focuses on Pan-Asian Fusion cuisine"""
)

dessert_sample = SingleTurnSample(
    user_input=kb_trace_dessert[0].get("user_input"),
    retrieved_contexts=kb_trace_dessert[0].get("retrieved_contexts"),
    response=kb_trace_dessert[0].get("response"),
    reference="Yes, Rice & Spice does have Mango Sticky Rice on their menu."
)

# Run evaluation
dataset = EvaluationDataset(samples=[wings_sample, dessert_sample])
kb_results = evaluate(dataset=dataset, metrics=metrics)

In [ ]:
kb_results


#### Custom Metrics

Based on our usecase, we want to create some custom metrics:

<div style="text-align:left">
    <img src="images/metrics_all.png" width="65%" />
</div>



We create custom metrics for specific aspects:
- `Request Completeness`: Evaluates if the agent fulfills all parts of the request
- `Brand Voice Metric`: Assesses if communication aligns with desired tone
- `Recommendations`: Uses a rubric to evaluate how well the agent handles food item requests



In [ ]:
from ragas.metrics import AspectCritic

# Metric to evaluate if the AI fulfills all human requests completely
request_completeness = AspectCritic(
    name="Request Completeness",
    llm=evaluator_llm,
    definition=(
        "Return 1 if the agent completely fulfills all the user requests with no omissions. "
        "otherwise, return 0."
    ),
)

# Metric to assess if the AI's communication aligns with the desired brand voice
brand_tone = AspectCritic(
    name="Brand Voice Metric",
    llm=evaluator_llm,
    definition=(
        "Return 1 if the AI's communication is friendly, approachable, helpful, clear, and concise; "
        "otherwise, return 0."
    ),
)

In [ ]:
queries = [
    """If you serve mushroom risotto at The Coastal Bloom? If so then book a 
    table for Anna, 2 people at 7pm on the 5th of August 2025.""",
    "Can you check my previous booking? Was it for 2 or 4 people?"
]

samples = []
session_id = str(uuid.uuid1())
for query in queries:
    agent_answer, traces = invokeAgent(query, session_id)
    print(agent_answer)
    ragas_messages_trace = convert_to_ragas_messages(traces)
    samples.append(MultiTurnSample(user_input=ragas_messages_trace))

result = evaluate(
    dataset=EvaluationDataset(samples=samples),
    metrics=[request_completeness, brand_tone],
)

In [ ]:
result

In [ ]:
from ragas.metrics import RubricsScore

rubrics = {
    "score0_description": (
        "The food item is not present in the menu and no recommendation was made."
    ),
    "score1_description": (
        "The item requested by the customer is not present in the menu and a recommendation was provided."
    ),
    "score2_description": (
        "No recommendation needed. The food item asked is is present in the menu."
    ),
    "score3_description": (
        "No recommendation needed. The request was not about a food item."
    ),
}

recommendations = RubricsScore(rubrics=rubrics, llm=evaluator_llm, name="Recommendations")

In [ ]:
queries = [
    "Do you serve Chicken Wings at The Coastal Bloom?",
    "Does Rice & Spice in San Francisco have Mango Sticky Rice in the menu?",
    "Does Ember & Vine have anything with chicken in the menu?",
    "Do you have Indian food at Bistro Parisienne?",
    "Where is the Ocean Harvest located?"
]
samples = []
answers = []

for query in queries:
    session_id:str = str(uuid.uuid1())
    agent_answer, traces = invokeAgent(query, session_id)
    ragas_messages_trace = convert_to_ragas_messages(traces)
    answers.append(agent_answer)
    samples.append(MultiTurnSample(user_input=ragas_messages_trace))

result = evaluate(
    dataset=EvaluationDataset(samples=samples),
    metrics=[recommendations],
)

results = result.to_pandas()
results['queries'] = queries
results['output'] = answers

In [ ]:
results[["queries", "output", "Recommendations"]]

In [ ]:
answers

### Cleanup (Optional but recommended)

In [ ]:
# Clean up resources
# Knowledge Base
kb.delete_kb(kb_name=knowledge_base_name, delete_s3_bucket=True, delete_iam_roles_and_policies=True, delete_aoss=True)

# Agent and Lambda action group
agents.delete_lambda(lambda_function_name=lambda_name, delete_role_flag=True)
agents.delete_agent(agent_name=agent_name, delete_role_flag=True, verbose=True)